# Pan-barley transcriptome data wrangling
## 1. Subset 1:1x20 Orthogroups
## 2. Create metatable for all accessions
## 3. Divide TPM table into accession-sets

In [2]:
library(tidyverse)
# import tables 
getwd()
tpm <- read.table("PanBaRT20_tpm_genes.csv", sep = ",", row.names = 1, header = TRUE, stringsAsFactors = TRUE)
hogs <- read.table("GsRTD_and_PanBaRT20_match.tsv", sep = "\t", header = TRUE, stringsAsFactors = TRUE)
head(tpm, 2)
dim(tpm)
dim(hogs)
head(hogs, 2)

,Akashinriki_Ca1,Akashinriki_Ca2,Akashinriki_Ca3,Akashinriki_Co1,Akashinriki_Co2,Akashinriki_Co3,Akashinriki_In1,Akashinriki_In2,Akashinriki_In3,Akashinriki_Ro1,⋯,RGTPlanet_Co3,RGTPlanet_In1,RGTPlanet_In2,RGTPlanet_In3,RGTPlanet_Ro1,RGTPlanet_Ro2,RGTPlanet_Ro3,RGTPlanet_Sh1,RGTPlanet_Sh2,RGTPlanet_Sh3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1H00001,4.007573,3.927487,3.833363,19.69449,26.30955,25.0926,4.080795,3.197684,6.416916,0.550499,⋯,18.30323,4.690004,4.166625,5.417613,1.272258,1.170904,1.360069,30.31152,27.36412,26.2589
chr1H00002,130.423413,95.909839,127.868675,606.02149,665.34842,636.3792,116.085727,77.322687,175.883190,16.129595,⋯,455.50814,101.854365,101.949697,130.543577,34.625894,32.760388,36.347136,888.63900,716.14744,670.2170


[1] 79580   297

[1] 2275093       9

,PanBaRT20_gene_id,PanBaRT20_transcript_id,PSVCP20_pos_strand,GsRTD_gene_id,GsRTD_transcript_id,Source_data,Genotype,Pan_category,PanBaRT20_DE_tissue
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,chr1H00001,chr1H00001.1,chr1H:49468-50716:-,HOR13942_chr1HG04824,HOR13942_chr1HG04824.1,RNA,HOR13942,shell;single,Co;In
2,chr1H00001,chr1H00001.1,chr1H:49468-50716:-,Barke_chr1HG02204,Barke_chr1HG02204.1,RNA,Barke,shell;single,Co;In


## 1. Subset 1:1x20 Orthogroups

In [3]:
orts <- hogs %>% filter(Pan_category == "core;single") %>% 
                 select(PanBaRT20_gene_id, PanBaRT20_transcript_id, GsRTD_gene_id, Genotype) %>% 
                 distinct()
glimpse(orts)
singlecopy <- orts %>% select(PanBaRT20_gene_id) %>% distinct()
glimpse(singlecopy)
# Save
getwd()
write.csv(singlecopy, file = "Singlecopy_PanBaRT20gene_all.csv", row.names = FALSE)

Rows: 1,046,822
Columns: 4
$ PanBaRT20_gene_id       <fct> chr1H00008, chr1H00008, chr1H00008, chr1H00008…
$ PanBaRT20_transcript_id <fct> chr1H00008.1, chr1H00008.2, chr1H00008.2, chr1…
$ GsRTD_gene_id           <fct> GoldenPromise_chr1HG04348, HOR10350_chr1HG0430…
$ Genotype                <fct> GoldenPromise, HOR10350, B1K-04-12, Igri, Hock…
Rows: 13,680
Columns: 1
$ PanBaRT20_gene_id <fct> chr1H00008, chr1H00009, chr1H00015, chr1H00045, chr1…


In [4]:
# Subset tpm table with 20let-HOGs translated to PanBaRT20_gene_id-s
tpm_geneID <- tpm %>% rownames_to_column("PanBaRT20_gene_id")
tpm_filt <- inner_join(singlecopy, tpm_geneID, by = "PanBaRT20_gene_id") %>% 
            column_to_rownames("PanBaRT20_gene_id")
dim(tpm_filt)
head(tpm_filt, 2)

[1] 13680   297

,Akashinriki_Ca1,Akashinriki_Ca2,Akashinriki_Ca3,Akashinriki_Co1,Akashinriki_Co2,Akashinriki_Co3,Akashinriki_In1,Akashinriki_In2,Akashinriki_In3,Akashinriki_Ro1,⋯,RGTPlanet_Co3,RGTPlanet_In1,RGTPlanet_In2,RGTPlanet_In3,RGTPlanet_Ro1,RGTPlanet_Ro2,RGTPlanet_Ro3,RGTPlanet_Sh1,RGTPlanet_Sh2,RGTPlanet_Sh3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1H00008,25.032419,24.800915,22.762422,18.038997,20.022145,16.312609,35.54198,26.42026,46.36280,22.75497,⋯,20.71802,39.13962,34.75701,40.04513,31.46420,33.54113,32.56900,19.69352,22.09276,21.90693
chr1H00009,8.906746,7.228023,7.626372,9.318535,8.712761,7.861657,37.56954,31.57817,28.24304,12.39945,⋯,10.85052,41.86304,36.21400,34.64365,15.11164,15.19396,14.78536,16.97324,16.66048,18.16478


In [5]:
trans_tpm <- as.data.frame(t(tpm))
head(trans_tpm, 2)

,chr1H00001,chr1H00002,chr1H00003,chr1H00004,chr1H00005,chr1H00006,chr1H00007,chr1H00008,chr1H00009,chr1H00010,⋯,chr7H79571,chr7H79572,chr7H79573,chr7H79574,chr7H79575,chr7H79576,chr7H79577,chr7H79578,chr7H79579,chr7H79580
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Akashinriki_Ca1,4.007573,130.42341,0,0,1.808930,5.673198,27.76783,25.03242,8.906746,214.8217,⋯,2.001052,0.713503,1.853821,0.475589,0,10.898248,25.31995,2.112613,0.058038,0
Akashinriki_Ca2,3.927487,95.90984,0,0,1.464428,5.194460,21.77769,24.80091,7.228023,195.0607,⋯,1.397187,0.264392,1.566389,0.209979,0,8.656334,20.34395,1.324194,0.289481,0


## 2. Create metatable for all accessions

In [6]:
# Create metatable
##################
meta <- trans_tpm %>% rownames_to_column("ID") %>%
                        select(ID) %>%
                        mutate(Accession = str_extract(ID, regex("[a-z0-9]+", ignore_case = TRUE))) %>%
                        mutate(Tissue = str_sub(str_extract(ID, regex("_[a-z]+", ignore_case = TRUE)), -2)) %>%
                        mutate(Batch = as.integer(str_extract(ID, "(?<=_\\w{2})\\d$"))) %>%
                        mutate(Tissue = case_when(Tissue == "Ca" ~ "Caryopsis",
                                                  Tissue == "Co" ~ "Coleoptiles",
                                                  Tissue == "Sh" ~ "Shoot",
                                                  Tissue == "Ro" ~ "Root",
                                                  Tissue == "In" ~ "Inflorescence"))
head(meta, 2)
glimpse(meta)

# save joined metatable for all accessions together
getwd()
write.csv(meta, file = "PanBaRT20_geneTPM_meta.csv", row.names = FALSE)

# save project table with each accession mentioned once
meta_acc <- meta %>% select(Accession) %>% distinct()
dim(meta_acc)
write.table(meta_acc, file = "PanBaRT20_project_table.txt", row.names = FALSE)

# save accession-wise metatable
acc <- unique(meta$Accession)
for (ac in acc) {
    # For the special case of ZDM01467: filter out In2 sample
    if (ac == "ZDM01467") {
        subset_meta <- meta %>% 
            filter(ID != "ZDM01467_In2") %>% 
            filter(Accession == ac)
    } else {
        subset_meta <- meta %>% filter(Accession == ac)
    }
    filename <- paste0(ac, "_meta.csv")
    write.csv(subset_meta, file = filename, row.names = FALSE)
}

,ID,Accession,Tissue,Batch
,<chr>,<chr>,<chr>,<int>
1,Akashinriki_Ca1,Akashinriki,Caryopsis,1
2,Akashinriki_Ca2,Akashinriki,Caryopsis,2


Rows: 297
Columns: 4
$ ID        <chr> "Akashinriki_Ca1", "Akashinriki_Ca2", "Akashinriki_Ca3", "Ak…
$ Accession <chr> "Akashinriki", "Akashinriki", "Akashinriki", "Akashinriki", …
$ Tissue    <chr> "Caryopsis", "Caryopsis", "Caryopsis", "Coleoptiles", "Coleo…
$ Batch     <int> 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, …


[1] 20  1

## 3. Divide TPM table into accession-sets

In [12]:
# divide accessions into separate tpm tables and save them
getwd()
# create vector of accession types
acc <- unique(meta$Accession)
acc

# First, determine the set of genes that pass the filter across all accessions
common_filtered_genes <- Reduce(intersect, 
                                lapply(acc, function(name) {
                                    matching_cols <- grep(paste0("^", name), colnames(tpm_filt))
                                    subtable <- tpm_filt[, matching_cols, drop = FALSE]
                                    
                                    # Special case for "ZDM01467" to exclude "ZDM01467_In2" column
                                    if (name == "ZDM01467") {
                                        subtable <- subtable[, !colnames(subtable) %in% c("ZDM01467_In2"), drop = FALSE]
                                    }  
                                    
                                    idx <- rowSums(subtable >= 0.5) >= 2
                                    return(rownames(subtable)[idx])
                                }))

# Save the common_filtered_genes list to a CSV file
common <- data.frame(PanBaRT20_gene_id=common_filtered_genes)
common_single20 <- inner_join(common, orts, by = "PanBaRT20_gene_id") %>% 
                    select(PanBaRT20_gene_id, PanBaRT20_transcript_id, Genotype) %>%
                    distinct()
glimpse(common_single20)
write.csv(common_single20, file = "Singlecopy_transcript_to_gene_expressing.csv", row.names = FALSE, quote = FALSE)
only_geneid <-common_single20 %>% 
                select(PanBaRT20_gene_id) %>% 
                distinct()
write.csv(only_geneid, file = "Singlecopy_PanBaRT20geneID_expressing.csv", row.names = FALSE, quote = FALSE)

getwd()

# Extract subtables for matching column names
subtables <- lapply(acc, function(name) {
  matching_cols <- grep(paste0("^", name), colnames(tpm_filt))
  subtable <- tpm_filt[rownames(tpm_filt) %in% common_filtered_genes, matching_cols, drop = FALSE]
  
  # Special case for "ZDM01467" to exclude "ZDM01467_In2" column
  if (name == "ZDM01467") {
    subtable <- subtable[, !colnames(subtable) %in% c("ZDM01467_In2"), drop = FALSE]
  }  
  
  print(paste0(name, " filtered genes with >0.5 for min 2 samples across all samples:", dim(subtable)))
  
  # Save each subtable in a separate file
  file_name <- paste0("PanBaRT20_geneTPM_ort_filt_", name, ".csv")
  write.csv(subtable, file = file_name, row.names = TRUE)
})

[1] "Akashinriki"   "Barke"         "ZDM02064"      "ZDM01467"     
 [5] "B1K"           "GoldenPromise" "Hockett"       "HOR10350"     
 [9] "HOR13821"      "HOR13942"      "HOR21599"      "HOR3081"      
[13] "HOR3365"       "HOR7552"       "HOR8148"       "HOR9043"      
[17] "Igri"          "Morex"         "OUN333"        "RGTPlanet"

Rows: 1,046,107
Columns: 3
$ PanBaRT20_gene_id       <chr> "chr1H00008", "chr1H00008", "chr1H00008", "chr…
$ PanBaRT20_transcript_id <fct> chr1H00008.1, chr1H00008.2, chr1H00008.2, chr1…
$ Genotype                <fct> GoldenPromise, HOR10350, B1K-04-12, Igri, Hock…
[1] "Akashinriki filtered genes with >0.5 for min 2 samples across all samples:13652"
[2] "Akashinriki filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "Barke filtered genes with >0.5 for min 2 samples across all samples:13652"
[2] "Barke filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "ZDM02064 filtered genes with >0.5 for min 2 samples across all samples:13652"
[2] "ZDM02064 filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "ZDM01467 filtered genes with >0.5 for min 2 samples across all samples:13652"
[2] "ZDM01467 filtered genes with >0.5 for min 2 samples across all samples:14"   
[1] "B1K filtered genes with >0.5 for min 2 samples across all sample

In [9]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/vanda/miniconda3/envs/r/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.2 forcats_1.0.0   stringr_1.5.0   dplyr_1.1.2    
 [5] purrr_1.0.1     readr_2.1.4     tidyr_1.3.0     tibble_3.2.1   
 [9] ggplot2_3.4.3   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] pillar_1.9.0     compiler_4.2.3   base64enc_0.1-3  tools_4.2.3     
 [5] digest_0.6.31